# Red Teaming with Azure AI Project

This notebook demonstrates how to use PyRIT with Azure AI Project for red teaming.

## Prerequisites

- Azure AI Project configured
- Azure OpenAI deployment
- Environment variables configured in `code/.env`

## Setup: Load Environment Variables

In [ ]:
import dotenv
from pathlib import Path
import os

# Load environment variables from code/.env
env_path = Path('/workspaces/genai-red-teaming-accelerator/code/.env')
dotenv.load_dotenv(dotenv_path=env_path)

# Validate required environment variables
required_vars = [
    'AZURE_OPENAI_ENDPOINT',
    'OPENAI_CHAT_API_KEY',
    'OPENAI_CHAT_MODEL',
    'AZURE_SUBSCRIPTION_ID',
    'AZURE_RESOURCE_GROUP_NAME',
    'AZURE_PROJECT_NAME'
]

for var in required_vars:
    if not os.getenv(var):
        raise ValueError(f"Missing environment variable: {var}")

print("✓ Environment variables loaded successfully")



### AOAI example


In [ ]:
# Azure imports
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory

# Azure AI Project Information
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP_NAME"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}

# Instantiate your AI Red Teaming Agent
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=DefaultAzureCredential(), # required
    risk_categories=[ # optional, defaults to all four risk categories
        RiskCategory.Violence,
        RiskCategory.HateUnfairness,
        RiskCategory.Sexual,
        RiskCategory.SelfHarm
    ], 
    num_objectives=5, # optional, defaults to 10
)

In [ ]:
# Configuration for Azure OpenAI model
azure_openai_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_KEY"), #  not needed for entra ID based auth, use az login before running,
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

red_team_result = await red_team_agent.scan(target=azure_openai_config)

In [ ]:
# Define a simple callback function that simulates a chatbot
def simple_callback(query: str) -> str:
    # Your implementation to call your application (e.g., RAG system, chatbot)
    return "I'm an AI assistant that follows ethical guidelines. I cannot provide harmful content."

red_team_result = await red_team_agent.scan(target=simple_callback)   

In [ ]:
import httpx
from azure.ai.evaluation.red_team import AttackStrategy


# async def myapi_callback(query: str) -> str:
#     payload = {
#         "user_prompt": query,
#         "conversation_id": "PromptSendingOrchestrator"
#     }
#     async with httpx.AsyncClient() as client:
#         response =  await client.post(
#             "http://127.0.0.1:8000/chat",
#             json=payload,
#             headers={"Content-Type": "application/json"}
#         )
#         #response.raise_for_status()
#         print("COUCOUCOUCOUCOUICOUCOU")
#         print(response.json()["choices"][0]["message"]["content"])
#         return response.json()["choices"][0]["message"]["content"] or "error"

# #test the callback function
# response = await myapi_callback("What is the capital of France?")
# print(response)  # Should print the response from your FastAPI application


def myapi_callback(query: str) -> str:
    payload = {
        "user_prompt": query,
        "conversation_id": "PromptSendingOrchestrator"
    }
    with httpx.Client() as client:
        response = client.post(
            "http://127.0.0.1:8000/chat",
            json=payload,
            headers={"Content-Type": "application/json"}
        )
        # response.raise_for_status()
        print("COUCOUCOUCOUCOUICOUCOU")
        print("Red teaming agent question:" + query)
        print("target response: "+ response.json()["choices"][0]["message"]["content"])
        return response.json()["choices"][0]["message"]["content"] or "error"

red_team_result = await red_team_agent.scan(target=myapi_callback, attack_strategies=[ # optional
        AttackStrategy.EASY, 
        AttackStrategy.MODERATE,  
        AttackStrategy.DIFFICULT,
    ],)   

In [ ]:
from pyrit.prompt_target import OpenAIChatTarget, PromptChatTarget

# Create a PyRIT PromptChatTarget for an Azure OpenAI model
# This could be any class that inherits from PromptChatTarget
chat_target = OpenAIChatTarget(
    model_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
) 

red_team_result = await red_team_agent.scan(target=chat_target)

In [ ]:
# from azure.ai.evaluation.red_team import AttackStrategy

# # Run the red team scan with multiple attack strategies
# red_team_agent_result = await red_team_agent.scan(
#     target=http_prompt_target, # required
#     scan_name="Scan with many strategies", # optional, names your scan in Azure AI Foundry
#     attack_strategies=[ # optional
#         AttackStrategy.EASY, 
#         AttackStrategy.MODERATE,  
#         AttackStrategy.DIFFICULT,
#     ],
# )